In [1]:
import pandas as pd

data = pd.read_csv("test_sample.csv")
data

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X482,X483,X484,X485,X486,X487,X488,X489,X490,X491
0,-16.781245,-2.299639,2.834571,-3.429914,0.066055,3.237802,-1.475841,0.350851,1.514042,1.118977,...,-2.201003,-1.002506,-3.827071,-1.479428,1.687928,0.673657,-2.921616,-0.297979,-4.233414,-1.504973
1,144.217835,0.207456,-1.289857,-1.580399,0.184094,-0.571962,1.624120,-0.144839,-1.488017,0.304977,...,1.165621,-2.998770,0.593848,-1.988718,3.239820,2.049295,2.404606,1.287899,1.150844,1.943235
2,-39.995367,-3.920722,2.707313,2.708473,-1.696070,1.221613,-0.299125,-0.298759,1.249226,0.802289,...,-2.167977,-1.774521,0.933038,-1.566605,1.119626,-1.719677,-0.719524,0.175910,-0.214671,-1.297748
3,59.132745,-0.198643,0.013281,0.117357,1.677367,3.045817,2.420261,-1.906969,-2.640600,-0.216244,...,-4.716180,-0.867243,-2.428916,-3.062165,-2.708479,1.648705,-0.782718,-2.748888,0.180156,1.811144
4,96.656473,-1.099724,1.147331,0.812714,-2.907078,0.277026,-3.393299,-0.159854,1.346470,0.078340,...,0.046267,-1.461278,-0.084345,-0.782103,-3.329513,-0.577401,2.157871,-0.588979,3.872724,0.768246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,77.850007,2.104096,-1.248604,0.330295,-1.919274,-0.865977,1.608078,-0.604410,1.700897,0.166654,...,0.482970,-0.978344,1.270737,0.356059,0.271384,4.059669,-0.130864,1.205250,-0.772908,-4.529742
496,-22.291617,-0.328945,-0.470804,2.272173,-0.804383,-0.136693,0.375305,2.908604,-2.410563,0.234126,...,-4.037164,-3.792729,0.221865,-0.441192,-2.557122,3.305660,2.150265,1.606178,0.799617,-1.269073
497,-38.195955,0.396289,-2.413359,-0.185637,-1.281254,-0.567833,-0.321495,-0.516100,-1.306189,-3.595318,...,-2.155309,0.128205,-0.163682,0.613586,-2.147816,0.308230,2.114563,-2.780197,2.482973,0.881377
498,-13.878043,1.708772,0.703832,2.372240,-0.101372,-1.841642,-1.001718,-0.563625,-0.476306,-0.704367,...,-0.636694,6.990187,2.496294,0.745239,-1.184435,-3.037727,1.241552,1.406165,-1.258977,3.799395


In [2]:
from sklearn.linear_model import LinearRegression

X = data.drop('Y', axis=1)
Y = data['Y']

# Fit linear regression models with increasing number of regressors
r2_values = []
for j in range(2, 492):
    model = LinearRegression()
    model.fit(X.iloc[:, :j], Y)
    r2 = model.score(X.iloc[:, :j], Y)
    r2_values.append(r2)

# Find the smallest number of regressors making determination coefficient greater than 0.9
for i, r2 in enumerate(r2_values):
    if r2 > 0.9:
        n_orig = i + 2
        break

print("Smallest number of regressors making determination coefficient greater than 0.9: ", n_orig)

Smallest number of regressors making determination coefficient greater than 0.9:  325


In [3]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X)

# Calculate the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Find the smallest number of PCA factors making determination coefficient greater than 0.9
n_PCA = np.argmax(cumulative_variance_ratio > 0.9) + 1

# Fit a linear regression model using the selected number of PCA factors
pca_model = PCA(n_components=n_PCA)
X_pca = pca_model.fit_transform(X)
model = LinearRegression()
model.fit(X_pca, Y)
r2_pca = model.score(X_pca, Y)

# Calculate the model dimensionality reduction
dim_reduction = n_orig - n_PCA

print("Smallest number of PCA factors making determination coefficient greater than 0.9: ", n_PCA)
print("Determination coefficient using PCA factors: ", r2_pca)
print("Model dimensionality reduction: ", dim_reduction)

Smallest number of PCA factors making determination coefficient greater than 0.9:  253
Determination coefficient using PCA factors:  0.9043287746069584
Model dimensionality reduction:  72


In [4]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

data = pd.read_csv("test_sample.csv")

# Step 1
r2_scores = []
for i in range(2, 492):
    X = data.iloc[:, 1:i]
    y = data.iloc[:, 0]
    lr = LinearRegression().fit(X, y)
    y_pred = lr.predict(X)
    r2 = r2_score(y, y_pred)
    r2_scores.append(r2)

n_orig = np.argmax(np.array(r2_scores) > 0.9) + 2

# Step 2
pca = PCA()
X_pca = pca.fit_transform(data.iloc[:, 1:492])

n_pca = np.argmax(np.cumsum(pca.explained_variance_ratio_) > 0.9) + 1

model_dim_reduction = n_orig - n_pca


In [5]:
model_dim_reduction

73